In [25]:
# Imports
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-ITGhy3LPfyvYrxbBci8pT3BlbkFJP5k2rqmOZhAdrOncxu9X"
os.environ["SERPER_API_KEY"] = "22c46d0058e5749e043b579d50860e011f4358ba"
# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from pydantic import BaseModel, Field
from crewai_tools import tool
import json

In [26]:
url = 'https://www.sutterhealth.org/'

In [27]:
# Using Pydantic for structured output
class Queries(BaseModel):
    queries: list[str] = Field(..., title="List of queries generated by the agent.")

In [28]:
serper_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool(website_url=url)
website_search_rag_tool = WebsiteSearchTool()

In [29]:
# Information Specialist
information_specialist = Agent(
    role = 'Information Specialist',
    goal = 'Gather all information about a healthcare provider',
    backstory = 
        """You work for a digital healthcare company called as Healee.
        You are an expert in detailed research and data compilation.
        You have to find out all the important information possible about {client} healthcare provider.""",
    tools = [scrape_tool],
    verbose = True,
    allow_delegation=False
)

# Query Agent
search_query_generator = Agent(
  role='Search Query Generator',
  goal='Generate search queries to find similar healthcare providers companies to {client} based on provided criteria and gathered information',
  backstory='Proficient in creating effective search queries from given data.',
  allow_delegation=False, 
  verbose = True
)

# Researcher Agent function
def create_researcher_agent(query):
    return Agent(
        role='Internet Researcher',
        goal=f'Find similar healthcare provider companies using the query: {query}',
        backstory="""You work in the marketing team of a digital healthcare company.
                     You are adept at using online resources to find relevant information quickly.
                     You are looking for potential healthcare provider clients similar to {client}
                     that your company can pitch your platform to and sign them.""",
        tools=[serper_tool],
        allow_delegation=False
    )

# Final Consolidator Agent
consolidator = Agent(
    role='Result Consolidator',
    goal='Combine all the results from the researcher agents into a final list of healthcare companies.',
    backstory='You excel at synthesizing information from multiple sources into a clear, concise list which has the company name and also the website link.',
    verbose=True,
    allow_delegation=False
)


In [30]:
# Initial Tasks (Genrating the queries)

scrape_task = Task(
    description="""Scrape and extract all the relevant information about the company.""",
    agent=information_specialist,
    expected_output="""All the details about the {client} in a concise summary that can be used to build search queries based on the 
     search criteria of {search_criteria} to find similar companies.""")

generate_search_query_task = Task(
  description="""Generate succinct search queries based on the provided criteria: {search_criteria} and 
  information gathered by the Information Specialist. The each query must include all of the search criteria. Generate at max 4 queries.""",
  expected_output='A Python list of strings, where each string is a search query.',
  agent=search_query_generator,
  output_json = Queries
)

# search_task = Task(
#     description="""Search the internet using different queries provided by the Search Query Generator agent to find {quantity} companies 
#     that the marketing team can pitch the Healee platform to. Provide a list of these companies, including a brief description of each 
#     company and a link to their website.""",
#     agent=researcher,
#     expected_output="A list of potential companies including brief descriptions and website links."
# )

In [31]:
# Define the crew with the initial tasks
initial_crew = Crew(
    agents=[information_specialist, search_query_generator],
    tasks=[scrape_task, generate_search_query_task],
    process=Process.sequential  # Ensure sequential execution for the initial tasks
)

2024-07-02 19:17:27,691 - 8667892736 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [32]:
# Test code
# search_queries = initial_crew.kickoff(inputs={'client': 'Sutter Health', 'search_criteria': 'location and speciality'})
# search_queries
# json_queries = json.loads(search_queries)
# json_queries['queries']

In [36]:
# generating the queries till we get the correct format
# setting max retries so that the system does not get stuck in an infinite loop
max_retries = 3
retry_count = 0
search_queries = None

while retry_count < max_retries:
    search_queries = initial_crew.kickoff(inputs={'client': 'Sutter Health', 'search_criteria': 'provider speciality and location'})
    json_queries = json.loads(search_queries)
    search_queries = json_queries['queries']
    # Break the while loop if the format is correct
    if isinstance(search_queries, list) and all(isinstance(query, str) for query in search_queries):
        break
    
    # Else we will loop back and run the crew again
    retry_count += 1

if not search_queries:
    raise ValueError("Failed to generate search queries in the correct format after multiple retries.")



> Entering new CrewAgentExecutor chain...
Thought: It's time to gather information about Sutter Health healthcare provider.

Action: Read website content

Action Input: {"url": "https://www.sutterhealth.org/" 

Sutter Health | Doctors and Hospitals | Northern California
Error was Detected
 						Your browser doesn't support JavaScript code, or you have disabled JavaScript.
Jump to content
Set Your Location
Sign in or Enroll
Set Your LocationSet Your Location
Find a Doctor
Find a Location
Donate Now
Patient Login
Sign In to My Health Online
Enroll in My Health Online
Learn About My Health Online
Enter a search term:
View all resultsfor 
Enter City or ZipGeolocateMake this my locationOpen 													Choose a medical group or hospital
Clear my location
Change LocationX
Sutter Health
BOOK APPOINTMENT
MENUOpen Menu
Book Appointment
Walk-In Care
In-person care for minor illnesses.
Urgent Care
In-person care for moderate illnesses and injuries.
Video Visits
Virtual care with a Walk-In Car

In [37]:
# Create search tasks for each generated query
search_tasks = [
    Task(
        description=f"Search the internet using the query: '{query}' to find similar companies.",
        agent=create_researcher_agent(query),
        expected_output=f"A list of potential companies along with their website links found using the query: '{query}'"
    ) for query in search_queries
]

# Define the consolidate results task
consolidate_results_task = Task(
    description="Consolidate the list of potential companies from the researcher agents into a final list.",
    agent=consolidator,
    expected_output="A final list of potential companies including brief descriptions and website links"
)

In [38]:
# Define the final crew with search tasks and consolidation task
final_crew = Crew(
    agents=[consolidator] + [task.agent for task in search_tasks],
    tasks=search_tasks + [consolidate_results_task],
    # process=Process.sequential  # Parallel execution for search tasks
)

# Kickoff the final crew to execute the search and consolidation tasks
final_result = final_crew.kickoff()

2024-07-02 19:19:44,058 - 8667892736 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 


Search results: 


 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 


Search results: 


 


Search results: 


 


Search results: Title: Find a Doctor - Columbus Regional Health
Link: https://www.crh.org/find-a-doctor
Snippet: Find a doctor and compare in our helpful physician directory. Locate primary care providers, specialists and practitioners near Columbus Indiana.
---
Title: [PDF] CMS SPECIALTY CODES/HEALTHCARE PROVIDER TAXONOMY
Link: https://www.cms.gov/Medicare/Provider-Enrollment-and-Certification/MedicareProviderSupEnroll/downloads/taxonomy.pdf
Snippet: This table reflects Medicare Specialty Codes as of April 1, 2003. This table reflects Healthcare Provider Taxonomy Codes (HPTC) effective July 1 ...
---
Title: 31 – Physician - Indiana Medicaid: Providers - IN.gov
Link: https://www.in.gov/medicaid/providers/provider-enrollment/complete-an-ihcp-provider-enrollment-application/31-physician/
Snippet: The Indiana H

In [39]:
print(final_result)

**

[
  {"Name": "Sutter Health", "Website Link": "https://www.sutterhealth.org/"},
  {"Name": "Kaiser Permanente", "Website Link": "https://www.kaiserpermanente.org/"},
  {"Name": "Dignity Health", "Website Link": "https://www.dignityhealth.org/"},
  {"Name": "UC San Francisco Medical Center (UCSF)", "Website Link": "https://www.ucsf.edu/medical-center/"},
  {"Name": "Stanford Health Care", "Website Link": "https://www.stanfordhealthcare.org/"},
  {"Name": "Cedars-Sinai Medical Center", "Website Link": "https://www.cedars-sinai.org/"},
  {"Name": "John Muir Health", "Website Link": "https://www.johnmuirhealth.com/"},
  {"Name": "UC San Diego Health", "Website Link": "https://health.ucsd.edu/"},
  {"Name": "Kaiser Permanente Southern California", "Website Link": "https://www.kaiserpermanente.org/service/northern-california-southern-california"},
]

Please let me know if this meets your expectations!
